In [ ]:
import pandas as pd
import pickle

import ipywidgets as widgets
from ipywidgets import interact

from src.data.data_load import load_tables, load_online_instance, load_distances
from src.data.solution_load import load_solution_dfs
from src.utils.filtering import flexible_filter
from src.utils.plotting import (
    plot_metrics_comparison, 
    plot_gantt_labors_by_driver, 
    plot_service_driver_distance,
    plot_gantt_by_services, 
    plot_gantt_by_drivers
)
from src.data.metrics import collect_results_to_df, compute_metrics_with_moves, get_day_plotting_df
from src.config.experimentation_config import *
from src.config.config import *

data_path = '../data'

instance = 'instRD4'        # Options: ['instAS1', 'instRS1']

distance_type = 'osrm'              # Options: ['osrm', 'manhattan']
dist_method = 'haversine'      # Options: ['precalced', 'haversine']

optimization_obj = 'driver_distance'

directorio_df, labors_raw_df, cities_df, duraciones_df, valid_cities = load_tables(data_path, generate_labors=False)
labors_real_df, labors_static_df, labors_dynamic_df = load_online_instance(data_path, instance, labors_raw_df)
dist_dict = load_distances(data_path, distance_type, instance, dist_method)

fechas = fechas_map(instance)

metricas = ['vt_count', 'num_drivers', 'driver_extra_time', 'driver_move_distance']

In [2]:
from src.data.data_load import upload_ONLINE_static_solution
# labors_algo_static_df, moves_algo_static_df, postponed_algo_static = upload_ONLINE_static_solution(
labors_algo_static_df, moves_algo_static_df, postponed_algo_static = upload_ONLINE_static_solution(
    data_path, 
    instance, 
    dist_method, 
    optimization_obj
)

# Cargar resultados

In [3]:
df = labors_real_df
# df = df[df['city'] != 'ALL']
flexible_filter(df,
                labor_id='306903')

,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,...,start_address_id,start_address_point,end_address_id,end_address_point,city,address_id,address_point,address_name,labor_sequence,date
494,214133,306903,6.0,General Mechanics,GENERAL_MECHANICS,2026570.0,2025-02-19 12:56:22.748000-05:00,2026-01-05 13:02:00-05:00,2026-01-05 07:06:00-05:00,,...,87960.0,POINT (-74.776479 10.8865373),87960.0,POINT (-74.776479 10.8865373),126,59476.0,POINT (-74.7919853 10.9101997),Lubricentro La Hormiga Soledad,0,2026-01-05


In [4]:
flexible_filter(labors_raw_df,
                service_id='214133')

,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,...,paying_customer,state_service,start_address_id,start_address_point,end_address_id,end_address_point,city,address_id,address_point,address_name
87938,214133,306903,6.0,General Mechanics,GENERAL_MECHANICS,2026570.0,2025-02-19 12:56:22.748000-05:00,2025-02-19 13:02:00-05:00,2025-02-20 07:06:00-05:00,,...,45184.0,COMPLETED,87960.0,POINT (-74.776479 10.8865373),87960.0,POINT (-74.776479 10.8865373),126,59476.0,POINT (-74.7919853 10.9101997),Lubricentro La Hormiga Soledad
87939,214133,307018,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,90004.0,2025-02-19 17:00:09.608000-05:00,2025-02-20 07:06:00-05:00,2025-02-20 08:33:00-05:00,14946,...,45184.0,COMPLETED,87960.0,POINT (-74.776479 10.8865373),87960.0,POINT (-74.776479 10.8865373),126,56474.0,POINT (-74.79419462825471 10.906019892143393),Casa


In [5]:
df = labors_real_df
# df = df[df['city'] != 'ALL']
flexible_filter(df,
                service_id='214133')

,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,...,start_address_id,start_address_point,end_address_id,end_address_point,city,address_id,address_point,address_name,labor_sequence,date
494,214133,306903,6.0,General Mechanics,GENERAL_MECHANICS,2026570.0,2025-02-19 12:56:22.748000-05:00,2026-01-05 13:02:00-05:00,2026-01-05 07:06:00-05:00,,...,87960.0,POINT (-74.776479 10.8865373),87960.0,POINT (-74.776479 10.8865373),126,59476.0,POINT (-74.7919853 10.9101997),Lubricentro La Hormiga Soledad,0,2026-01-05
495,214133,307018,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,90004.0,2025-02-19 17:00:09.608000-05:00,2026-01-05 07:06:00-05:00,2026-01-05 08:33:00-05:00,14946,...,87960.0,POINT (-74.776479 10.8865373),87960.0,POINT (-74.776479 10.8865373),126,56474.0,POINT (-74.79419462825471 10.906019892143393),Casa,1,2026-01-05


In [6]:
labors_hist_df, moves_hist_df, postponed_labors_hist = load_solution_dfs(
    data_path, 
    instance, 
    dist_method,
    algorithm='hist',
    include_all_city=True
)

labors_algo_baseline_df, moves_algo_baseline_df, postponed_labors_OFFLINE = load_solution_dfs(
    data_path, 
    instance, 
    dist_method,
    algorithm='OFFLINE',
    include_all_city=True
)

labors_algo_static_df, moves_algo_static_df, postponed_labors_ONLINE_static = load_solution_dfs(
    data_path,
    instance,
    dist_method,
    algorithm='ONLINE_static',
    include_all_city=True
)

# labors_algo_INSERT_df, moves_algo_INSERT_df, postponed_labors_INSERT = load_solution_dfs(
#     data_path,
#     instance,
#     dist_method,
#     algorithm='INSERT',
#     include_all_city=True
# )

# labors_algo_INSERT_BUFFER_df, moves_algo_INSERT_BUFFER_df, postponed_labors_INSERT_BUFFER = load_solution_dfs(
#     data_path,
#     instance,
#     dist_method,
#     algorithm='INSERT_BUFFER',
#     include_all_city=True
# )

# labors_algo_REACT_df, moves_algo_REACT_df, postponed_labors_REACT_BUFFER_0 = load_solution_dfs(
#     data_path, 
#     instance, 
#     dist_method,
#     algorithm='REACT_BUFFER_0',
#     include_all_city=True
# )

# labors_algo_REACT_BUFFER_df, moves_algo_REACT_BUFFER_df, postponed_labors_REACT_BUFFER = load_solution_dfs(
#     data_path, 
#     instance, 
#     dist_method,
#     algorithm='REACT_BUFFER',
#     include_all_city=True
# )

algorithms = [
    {
        "name": "Histórico",
        "labors_df": labors_hist_df,
        "moves_df": moves_hist_df,
        "type": "historic",
        "color": "#C0392B",  # 🔴 Deep crimson red – “you were here”
        "visible": True,
    },
    {
        "name": "Offline",
        "labors_df": labors_algo_baseline_df,
        "moves_df": moves_algo_baseline_df,
        "type": "algorithm",
        "color": "#2980B9",  # 🔵 Royal blue – “best possible solution”
        "visible": True,
    },
    # {
    #     "name": "INSERT",
    #     "labors_df": labors_algo_INSERT_df,
    #     "moves_df": moves_algo_INSERT_df,
    #     "type": "algorithm",
    #     "color": "#27AE60",  # 🟢 Vibrant green – “first smart improvement”
    #     "visible": True,
    # },
    # {
    #     "name": "INSERT_BUFFER",
    #     "labors_df": labors_algo_INSERT_BUFFER_df,
    #     "moves_df": moves_algo_INSERT_BUFFER_df,
    #     "type": "algorithm",
    #     "color": "#16A085",
    #     "visible": True,
    # },
    # {
    #     "name": "REACT",
    #     "labors_df": labors_algo_REACT_df,
    #     "moves_df": moves_algo_REACT_df,
    #     "type": "algorithm",
    #     "color": "#E67E22",  # 🟠 Bold orange – “fast adaptation”
    #     "visible": True,
    # },
    # {
    #     "name": "REACT_BUFFER",
    #     "labors_df": labors_algo_REACT_BUFFER_df,
    #     "moves_df": moves_algo_REACT_BUFFER_df,
    #     "type": "algorithm",
    #     "color": "#9B59B6",  # 🟣 Electric purple – “intelligent batching”
    #     "visible": True,
    # }
]

# Resultados

In [11]:
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from src.utils.utils import get_city_name_from_code
# from src.alpha_tuning_utils import add_aggregated_totals

def plot_metrics_comparison_dynamic(
    algorithms,
    city,
    metricas,
    dist_dict,
    fechas,
    metrics_with_gap=None,  # <-- NEW
    group_by=None,
    xaxis_mode="date"
):
    """Compares any number of algorithms/historic datasets dynamically, 
    with optional relative gap display vs baseline 'Histórico'."""

    # Default empty list if not provided
    metrics_with_gap = metrics_with_gap or []

    all_metrics = pd.DataFrame()

    # Compute metrics for each algorithm
    metrics_by_algo = {}
    for algo in algorithms:
        df = compute_metrics_with_moves(
            algo["labors_df"], algo["moves_df"], fechas, dist_dict,
            workday_hours=8,
            city=city,
            assignment_type=algo["type"],
            skip_weekends=False,
            dist_method='haversine'
        )
        metrics_by_algo[algo["name"]] = df

        df_altered = df.copy()
        df_altered['algo'] = algo['name']
        all_metrics = pd.concat([all_metrics, df_altered])

    all_metrics.to_csv('./results_summary.csv', index=True)

    if isinstance(metricas, str):
        metricas = [metricas]

    # Prepare x-axis
    x_vals = pd.to_datetime(metrics_by_algo[algorithms[0]["name"]]["day"]).dt.strftime("%Y-%m-%d")

    # Create one figure per metric
    for metrica in metricas:
        fig = make_subplots(
            rows=1, cols=2,
            column_widths=[0.35, 0.65],
            subplot_titles=["Totales", "Serie diaria"]
        )

        totals = {
            algo["name"]: metrics_by_algo[algo["name"]][metrica].sum()
            for algo in algorithms
        }

        # Get baseline (Histórico)
        baseline_val = totals.get("Histórico", None)

        # --- Bar plot (left) ---
        for algo in algorithms:
            if algo.get("visible", True):
                val = totals[algo["name"]]

                # Compute percentage or absolute gap vs baseline
                if (
                    baseline_val is not None 
                    and baseline_val != 0 
                    and metrica in metrics_with_gap
                    and algo["name"] != "Histórico"
                ):
                    abs_gap = val - baseline_val
                    perc_gap = (abs_gap / baseline_val) * 100
                    text = f"{val:.0f}<br>{perc_gap:+.1f}%"
                else:
                    text = f"{val:.0f}"

                fig.add_trace(go.Bar(
                    x=[algo["name"]],
                    y=[val],
                    marker_color=algo["color"],
                    name=algo["name"],
                    text=[text],
                    textposition="auto"
                ), row=1, col=1)

        # --- Time series (right) ---
        for algo in algorithms:
            if algo.get("visible", True):
                dash_style = (
                    "solid" if algo["type"] == "historic"
                    else "dashdot" if "react" in algo["name"].lower()
                    else "dash"
                )
                fig.add_trace(go.Scatter(
                    x=x_vals,
                    y=metrics_by_algo[algo["name"]][metrica],
                    mode="lines+markers",
                    name=algo["name"],
                    line=dict(color=algo["color"], dash=dash_style, width=2),
                    marker=dict(size=7)
                ), row=1, col=2)

        # --- Layout ---
        titles = {'vt_count': 'Número servicios Vehicle Transportation',
                  'num_drivers': 'Número conductores',
                  'driver_extra_time': 'Tiempo extra', 
                  'driver_move_distance': 'Distancia en vacío'
                  }
        fig.update_layout(
            height=500, width=1100,
            title=dict(
                text=f"{titles[metrica]} — {get_city_name_from_code(city)}",
                x=0.5, xanchor="center"
            ),
            legend=dict(
                orientation="h",
                yanchor="top",
                y=-0.25,
                xanchor="center",
                x=0.5,
                font=dict(size=11)
            ),
            plot_bgcolor="white",
            margin=dict(t=100, b=120)
        )

        fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor="lightgray")
        fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor="lightgray")

        fig.show()


def plot_metrics_comparison_interactive_dynamic(
    algorithms, cities, metricas, dist_dict, fechas, metrics_with_gap=None
):
    """Fully dynamic interactive version for any number of algorithms."""

    city_dropdown = widgets.Dropdown(
        options=['ALL'] + sorted(list(set(cities))),
        value='ALL',
        description="Ciudad:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="300px")
    )

    # dynamically create visibility checkboxes
    algo_checks = {
        algo["name"]: widgets.Checkbox(value=algo.get("visible", True), description=algo["name"])
        for algo in algorithms
    }

    def _plot(city, **vis_flags):
        algo_copy = [{**algo, "visible": vis_flags[algo["name"]]} for algo in algorithms]
        plot_metrics_comparison_dynamic(
            algo_copy,
            city=city,
            metricas=metricas,
            dist_dict=dist_dict,
            fechas=fechas,
            metrics_with_gap=metrics_with_gap
        )

    interact(
        _plot,
        city=city_dropdown,
        **algo_checks
    )

metrics_with_gap = [
    "total_distance",
    "driver_move_distance",
    "labor_extra_time",
    "driver_extra_time"
]

plot_metrics_comparison_interactive_dynamic(
    algorithms,
    cities=valid_cities,
    metricas=metricas,
    dist_dict=dist_dict,
    fechas=(fechas[0], fechas[-1]),
    metrics_with_gap=metrics_with_gap
)


interactive(children=(Dropdown(description='Ciudad:', layout=Layout(width='300px'), options=('ALL', '1', '1004…

# Validación de soluciones

In [5]:
from src.algorithms.solution_validator import validate_all_algorithms
validation_summary, moves_issues = validate_all_algorithms(
    algorithms[1:], 
    dist_method=dist_method,
    dist_dict=dist_dict
    )
display(validation_summary)



🚀 Validating Offline ...
🧑‍🔧 Checked 41 drivers, found 0 overlaps
✅ Validated 241 drivers, 3015 rows, 2 inconsistencies

🚀 Validating INSERT ...
🧑‍🔧 Checked 41 drivers, found 101 overlaps
✅ Validated 244 drivers, 3075 rows, 2 inconsistencies

🚀 Validating INSERT_BUFFER ...
🧑‍🔧 Checked 38 drivers, found 41 overlaps
✅ Validated 207 drivers, 1896 rows, 0 inconsistencies

🚀 Validating REACT ...
🧑‍🔧 Checked 42 drivers, found 0 overlaps
✅ Validated 230 drivers, 3069 rows, 0 inconsistencies

🚀 Validating REACT_BUFFER ...
🧑‍🔧 Checked 42 drivers, found 0 overlaps
✅ Validated 231 drivers, 3045 rows, 0 inconsistencies

📊 Validation Summary:


,algorithm,n_labors,n_moves,overlaps,move_inconsistencies,drivers_checked,rows_checked,pairs_checked,distance_calls,inconsistencies
0,Offline,1180,3015,0,2,241,3015,3015,3015,2
1,INSERT,1137,3075,101,2,244,3075,3075,3075,2
2,INSERT_BUFFER,691,1896,41,0,207,1896,1896,1896,0
3,REACT,1051,3069,0,0,230,3069,3069,3069,0
4,REACT_BUFFER,1050,3045,0,0,231,3045,3045,3045,0


# Debugging

## Initial sanity checks

In [8]:
labors_1 = labors_algo_baseline_df[labors_algo_baseline_df['city'] != 'ALL'].dropna(subset=['actual_start']).copy()
labors_2 = labors_algo_INSERT_BUFFER_df[labors_algo_INSERT_BUFFER_df['city'] != 'ALL'].dropna(subset=['actual_start']).copy()

moves_1 = moves_algo_baseline_df[moves_algo_baseline_df['city'] != 'ALL'].dropna(subset=['actual_start']).copy()
moves_2 = moves_algo_INSERT_BUFFER_df[moves_algo_INSERT_BUFFER_df['city'] != 'ALL'].dropna(subset=['actual_start']).copy()

## Individual checks

In [11]:
city = '844'
fecha = '2026-01-05'

In [14]:
drop_columns = ['labor_type', 'labor_name', 'shop', 'alfred', 'labor_price', 
                'labor_created_at', 'labor_start_date', 'labor_end_date', 
                'client_type', 'paying_customer', 'state_service', 'labor_category',
                'start_address_id', 'end_address_id', 'date', 'address_id', 'address_point']
column_order = ['service_id', 'labor_id', 'created_at', 'schedule_date', 'actual_start', 'actual_end', 'map_start_point', 'map_end_point', 'city', 'assigned_driver', 'dist_km']

## NUMBER OF LABORS

In [17]:
filtered_labors_baseline = flexible_filter(labors_algo_baseline_df,
                city=city,
                schedule_date=fecha)

filtered_labors_static = flexible_filter(labors_algo_static_df,
                city=city,
                schedule_date=fecha)

filtered_labors_INSERT = flexible_filter(labors_algo_INSERT_df,
                city=city,
                schedule_date=fecha)

filtered_labors_INSERT_BUFFER = flexible_filter(labors_algo_INSERT_BUFFER_df,
                city=city,
                schedule_date=fecha)

filtered_labors_REACT = flexible_filter(labors_algo_REACT_df,
                city=city,
                schedule_date=fecha)

filtered_labors_REACT_BUFFER = flexible_filter(labors_algo_REACT_BUFFER_df,
                city=city,
                schedule_date=fecha)

filtered_moves_baseline = flexible_filter(moves_algo_baseline_df,
                city=city,
                schedule_date=fecha)

filtered_moves_INSERT = flexible_filter(moves_algo_INSERT_df,
                city=city,
                schedule_date=fecha)

filtered_moves_INSERT_BUFFER = flexible_filter(moves_algo_INSERT_BUFFER_df,
                city=city,
                schedule_date=fecha)

filtered_moves_REACT = flexible_filter(moves_algo_REACT_df,
                city=city,
                schedule_date=fecha)

filtered_moves_REACT_BUFFER = flexible_filter(moves_algo_REACT_BUFFER_df,
                city=city,
                schedule_date=fecha)

print(f'-------- Labors')
print(f'Baseline:      {len(filtered_labors_baseline)}')
print(f'INSERT:        {len(filtered_labors_INSERT)}')
print(f'INSERT BUFFER: {len(filtered_labors_INSERT_BUFFER)}')
print(f'REACT:         {len(filtered_labors_REACT)}')
print(f'REACT BUFFER:  {len(filtered_labors_REACT_BUFFER)}')

print(f'\n-------- Moves')
print(f'Baseline:      {len(filtered_moves_baseline)}')
print(f'INSERT:        {len(filtered_moves_INSERT)}')
print(f'INSERT BUFFER: {len(filtered_moves_INSERT_BUFFER)}')
print(f'REACT:         {len(filtered_moves_REACT)}')
print(f'REACT BUFFER:  {len(filtered_moves_REACT_BUFFER)}')

-------- Labors
Baseline:      4
INSERT:        4
INSERT BUFFER: 2
REACT:         4
REACT BUFFER:  4

-------- Moves
Baseline:      12
INSERT:        12
INSERT BUFFER: 6
REACT:         6
REACT BUFFER:  6


## MISSING LABORS/MOVES IN 844 05-01-25

### Labors

In [19]:
flexible_filter(
    labors_algo_baseline_df.drop(
        columns=drop_columns
    )[column_order],
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])

,service_id,labor_id,created_at,schedule_date,actual_start,actual_end,map_start_point,map_end_point,city,dist_km
0,218033,311188,2026-01-03 08:22:06.741000-05:00,2026-01-05 07:00:00-05:00,2026-01-05 06:30:00-05:00,2026-01-05 07:17:09.137485-05:00,POINT (-73.11255469999999 7.117752199999999),POINT (-73.1192449 7.1288176),844,1.434861
1,218109,311287,2026-01-03 12:01:47.748000-05:00,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:59:36.286581-05:00,POINT (-73.0782802 7.0281457),POINT (-73.11350980000002 7.108425699999999),844,9.736518
2,218905,312140,2026-01-05 11:06:20.985000-05:00,2026-01-05 16:00:00-05:00,2026-01-05 15:30:00-05:00,2026-01-05 16:22:59.420799-05:00,POINT (-73.0975205 7.0649086),POINT (-73.1149525 7.109582199999998),844,5.326898
3,218944,312188,2026-01-05 13:44:35.523000-05:00,2026-01-05 18:00:00-05:00,2026-01-05 17:30:00-05:00,2026-01-05 18:29:36.286581-05:00,POINT (-73.11350980000002 7.108425699999999),POINT (-73.0782802 7.0281457),844,9.736518


In [18]:
flexible_filter(
    labors_algo_INSERT_BUFFER_df.drop(
        columns=drop_columns
    )[column_order], 
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])

,service_id,labor_id,created_at,schedule_date,actual_start,actual_end,map_start_point,map_end_point,city,dist_km
0,218033,311188,2026-01-03 08:22:06.741000-05:00,2026-01-05 07:00:00-05:00,2026-01-05 06:30:00-05:00,2026-01-05 07:17:09.137485-05:00,POINT (-73.11255469999999 7.117752199999999),POINT (-73.1192449 7.1288176),844,1.434861
1,218109,311287,2026-01-03 12:01:47.748000-05:00,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:59:36.286581-05:00,POINT (-73.0782802 7.0281457),POINT (-73.11350980000002 7.108425699999999),844,9.736518


In [20]:
flexible_filter(
    labors_algo_REACT_df.drop(
        columns=drop_columns
    )[column_order], 
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])

,service_id,labor_id,created_at,schedule_date,actual_start,actual_end,map_start_point,map_end_point,city,dist_km
0,218033,311188,2026-01-03 08:22:06.741000-05:00,2026-01-05 07:00:00-05:00,2026-01-05 06:30:00-05:00,2026-01-05 07:17:09.137485-05:00,POINT (-73.11255469999999 7.117752199999999),POINT (-73.1192449 7.1288176),844,1.434861
1,218109,311287,2026-01-03 12:01:47.748000-05:00,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:59:36.286581-05:00,POINT (-73.0782802 7.0281457),POINT (-73.11350980000002 7.108425699999999),844,9.736518
2,218905,312140,2026-01-05 11:06:20.985000-05:00,2026-01-05 16:00:00-05:00,2026-01-05 15:30:00-05:00,2026-01-05 16:22:59.420799-05:00,POINT (-73.0975205 7.0649086),POINT (-73.1149525 7.109582199999998),844,5.326898
3,218944,312188,2026-01-05 13:44:35.523000-05:00,2026-01-05 18:00:00-05:00,2026-01-05 17:30:00-05:00,2026-01-05 18:29:36.286581-05:00,POINT (-73.11350980000002 7.108425699999999),POINT (-73.0782802 7.0281457),844,9.736518


### Moves

In [24]:
flexible_filter(
    moves_algo_baseline_df, 
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])


,service_id,labor_id,labor_context_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
0,218033,311188,311188_free,FREE_TIME,FREE_TIME,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:23:45.091679-05:00,2026-01-05 06:23:45.091679-05:00,POINT (-73.1037471 7.0910492),POINT (-73.1037471 7.0910492),0.000000,0.0,844,2026-01-05
1,218033,311188,311188_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:23:45.091679-05:00,2026-01-05 06:30:00-05:00,POINT (-73.1037471 7.0910492),POINT (-73.11255469999999 7.117752199999999),3.124236,6.2,844,2026-01-05
2,218033,311188,311188_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:30:00-05:00,2026-01-05 07:17:09.137485-05:00,POINT (-73.11255469999999 7.117752199999999),POINT (-73.1192449 7.1288176),NaN,47.2,844,2026-01-05
3,218109,311287,311287_free,FREE_TIME,FREE_TIME,6412,2026-01-05 09:30:00-05:00,2026-01-05 07:17:09.137485-05:00,2026-01-05 08:35:51.307394-05:00,POINT (-73.1192449 7.1288176),POINT (-73.1192449 7.1288176),0.000000,78.7,844,2026-01-05
4,218109,311287,311287_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 09:30:00-05:00,2026-01-05 08:35:51.307394-05:00,2026-01-05 09:00:00-05:00,POINT (-73.1192449 7.1288176),POINT (-73.0782802 7.0281457),12.072438,24.1,844,2026-01-05
5,218109,311287,311287_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:59:36.286581-05:00,POINT (-73.0782802 7.0281457),POINT (-73.11350980000002 7.108425699999999),NaN,59.6,844,2026-01-05
6,218905,312140,312140_free,FREE_TIME,FREE_TIME,6412,2026-01-05 16:00:00-05:00,2026-01-05 09:59:36.286581-05:00,2026-01-05 15:19:41.939577-05:00,POINT (-73.11350980000002 7.108425699999999),POINT (-73.11350980000002 7.108425699999999),0.000000,320.1,844,2026-01-05
7,218905,312140,312140_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 16:00:00-05:00,2026-01-05 15:19:41.939577-05:00,2026-01-05 15:30:00-05:00,POINT (-73.11350980000002 7.108425699999999),POINT (-73.0975205 7.0649086),5.150504,10.3,844,2026-01-05
8,218905,312140,312140_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 16:00:00-05:00,2026-01-05 15:30:00-05:00,2026-01-05 16:22:59.420799-05:00,POINT (-73.0975205 7.0649086),POINT (-73.1149525 7.109582199999998),NaN,53.0,844,2026-01-05
9,218944,312188,312188_free,FREE_TIME,FREE_TIME,6412,2026-01-05 18:00:00-05:00,2026-01-05 16:22:59.420799-05:00,2026-01-05 17:29:35.443094-05:00,POINT (-73.1149525 7.109582199999998),POINT (-73.1149525 7.109582199999998),0.000000,66.6,844,2026-01-05


In [23]:
flexible_filter(
    moves_algo_REACT_df, 
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])

,service_id,labor_id,labor_context_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
0,218033,311188,311188_free,FREE_TIME,FREE_TIME,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:23:45.091679-05:00,2026-01-05 06:23:45.091679-05:00,POINT (-73.1037471 7.0910492),POINT (-73.1037471 7.0910492),0.000000,0.0,844,2026-01-05
1,218033,311188,311188_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:23:45.091679-05:00,2026-01-05 06:30:00-05:00,POINT (-73.1037471 7.0910492),POINT (-73.11255469999999 7.117752199999999),3.124236,6.2,844,2026-01-05
2,218033,311188,311188_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:30:00-05:00,2026-01-05 07:17:09.137485-05:00,POINT (-73.11255469999999 7.117752199999999),POINT (-73.1192449 7.1288176),NaN,47.2,844,2026-01-05
3,218109,311287,311287_free,FREE_TIME,FREE_TIME,6412,2026-01-05 09:30:00-05:00,2026-01-05 07:17:09.137485-05:00,2026-01-05 08:35:51.307394-05:00,POINT (-73.1192449 7.1288176),POINT (-73.1192449 7.1288176),0.000000,78.7,844,2026-01-05
4,218109,311287,311287_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 09:30:00-05:00,2026-01-05 08:35:51.307394-05:00,2026-01-05 09:00:00-05:00,POINT (-73.1192449 7.1288176),POINT (-73.0782802 7.0281457),12.072438,24.1,844,2026-01-05
5,218109,311287,311287_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:59:36.286581-05:00,POINT (-73.0782802 7.0281457),POINT (-73.11350980000002 7.108425699999999),NaN,59.6,844,2026-01-05


In [31]:
df = moves_algo_baseline_df
df = df[df['city'] != 'ALL']
df

,service_id,labor_id,labor_context_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
0,205621,297525,297525_free,FREE_TIME,FREE_TIME,10491,2026-01-05 07:30:00-05:00,2026-01-05 06:53:04.404846-05:00,2026-01-05 06:53:04.404846-05:00,POINT (-75.5948708 6.222133500000001),POINT (-75.5948708 6.222133500000001),0.000000,0.0,1,2026-01-05
1,205621,297525,297525_move,DRIVER_MOVE,DRIVER_MOVE,10491,2026-01-05 07:30:00-05:00,2026-01-05 06:53:04.404846-05:00,2026-01-05 07:00:00-05:00,POINT (-75.5948708 6.222133500000001),POINT (-75.5722612 6.2005726),3.463293,6.9,1,2026-01-05
2,205621,297525,297525_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,10491,2026-01-05 07:30:00-05:00,2026-01-05 07:00:00-05:00,2026-01-05 07:49:31.984874-05:00,POINT (-75.5722612 6.2005726),POINT (-75.56991479999999 6.2276503),NaN,49.5,1,2026-01-05
3,206697,298694,298694_free,FREE_TIME,FREE_TIME,31857,2026-01-05 08:00:00-05:00,2026-01-05 07:23:59.172324-05:00,2026-01-05 07:23:59.172324-05:00,POINT (-75.58290559999999 6.155980599999999),POINT (-75.58290559999999 6.155980599999999),0.000000,0.0,1,2026-01-05
4,206697,298694,298694_move,DRIVER_MOVE,DRIVER_MOVE,31857,2026-01-05 08:00:00-05:00,2026-01-05 07:23:59.172324-05:00,2026-01-05 07:30:00-05:00,POINT (-75.58290559999999 6.155980599999999),POINT (-75.5673676 6.1781754),3.006897,6.0,1,2026-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,215474,308380,308380_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-11 16:30:00-05:00,2026-01-11 15:55:44.773032-05:00,2026-01-11 16:00:00-05:00,POINT (-73.1037471 7.0910492),POINT (-73.11345413684005 7.107574257887976),2.126891,4.3,844,2026-01-11
11,215474,308380,308380_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-11 16:30:00-05:00,2026-01-11 16:00:00-05:00,2026-01-11 16:46:46.377629-05:00,POINT (-73.11345413684005 7.107574257887976),POINT (-73.1063319 7.1155142),NaN,46.8,844,2026-01-11
6,215495,308403,308403_free,FREE_TIME,FREE_TIME,6412,2026-01-11 13:00:00-05:00,2026-01-11 10:19:15.875810-05:00,2026-01-11 12:14:23.509259-05:00,POINT (-73.1134749 7.1084515),POINT (-73.1134749 7.1084515),0.000000,115.1,844,2026-01-11
7,215495,308403,308403_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-11 13:00:00-05:00,2026-01-11 12:14:23.509259-05:00,2026-01-11 12:30:00-05:00,POINT (-73.1134749 7.1084515),POINT (-73.181381 7.128081700000001),7.804090,15.6,844,2026-01-11


In [33]:
df = moves_algo_REACT_df
df = df[df['city'] != 'ALL']
df[df['service_id'] == '218944']

,service_id,labor_id,labor_context_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
9,218944,312188,312188_free,FREE_TIME,FREE_TIME,6412,2026-01-05 18:00:00-05:00,2026-01-05 16:22:59.420799-05:00,2026-01-05 17:29:35.443094-05:00,POINT (-73.1149525 7.109582199999998),POINT (-73.1149525 7.109582199999998),0.000000,66.6,nan,NaN
10,218944,312188,312188_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 18:00:00-05:00,2026-01-05 17:29:35.443094-05:00,2026-01-05 17:30:00-05:00,POINT (-73.1149525 7.109582199999998),POINT (-73.11350980000002 7.108425699999999),0.204641,0.4,nan,NaN
11,218944,312188,312188_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 18:00:00-05:00,2026-01-05 17:30:00-05:00,2026-01-05 18:29:36.286581-05:00,POINT (-73.11350980000002 7.108425699999999),POINT (-73.0782802 7.0281457),NaN,59.6,nan,NaN
